In [1]:
import os, csv, random, time

from __future__ import division
from matplotlib import  pyplot as plt
from scipy import stats, optimize
from collections import deque
from statsmodels.base.model import GenericLikelihoodModel
from scipy.stats import bernoulli,poisson,norm,expon
from multiprocessing import Process, Lock
# from pathos.multiprocessing import ProcessingPool as Pool

import numpy as np
import scipy.io as sio
import seaborn as sns

%matplotlib inline

In [5]:
global w

video_num = 30

video_score = np.random.uniform(-5,5,video_num)
resolution = 0.1
video_score = np.round(video_score/resolution)*resolution

pairs_truth = []

for i in range(len(video_score)-1):
    for j in range(i+1, len(video_score)):
        if video_score[i] > video_score[j]:
            pairs_truth.append((i,j))
        else:
            pairs_truth.append((j,i))
            
total_pairs = len(pairs_truth)

print 'Total number of pairs for %d videos is: ' %video_num, total_pairs

Total number of pairs for 30 videos is:  435


In [6]:
def update_w(pairs, w, w_old, i, eta):
    gradient = 0
    
    for pair in pairs:
        if i == pair[0]:
            out = -1
        elif i == pair[1]:
            out = 1  
        else:
            out = 0
            
        gradient -= out/(1/(np.exp(w_old[pair[1]]-w_old[pair[0]]))+1)
    
    w[i] += eta*gradient

def update_gradient(pairs, l, eta):
    w_old = w
    for i in range(len(w)):
        l.acquire()
        Process(target=update_w, args=(pairs, w, w_old, i, eta)).start()
        l.release()

def likelihood(pairs):
    l = 1
    for pair in pairs:
        l += np.log(1+np.exp(w[pair[1]]-w[pair[0]]))
    return l

In [7]:
w = np.zeros(video_num)

num_test_pairs = int(0.2* total_pairs)

pairs = [pairs_truth[i] for i in random.sample(range(total_pairs), num_test_pairs)]

results = []

like_prev = 100000

lock = Lock()

for i in range(1500):
    update_gradient(pairs, lock, 0.001)
    
    if i % 100 ==0:
        like =  likelihood(pairs_truth)
        like_prev = like
        print like
    results.append(w[0])

302.519023544
302.519023544


KeyboardInterrupt: 